# 数据库连接示例

本 notebook 演示如何连接到项目中的 MySQL 和 PostgreSQL 数据库。

## 1. 导入必要的库

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector
import psycopg2

## 2. MySQL 连接示例

In [ ]:
# 使用环境变量获取连接信息
mysql_config = {
    'host': os.getenv('MYSQL_HOST', 'mysql-db'),
    'port': int(os.getenv('MYSQL_PORT', 3306)),
    'database': os.getenv('MYSQL_DATABASE', 'sqlExpert'),
    'user': os.getenv('MYSQL_USERNAME', 'sqluser'),
    'password': os.getenv('MYSQL_PASSWORD', 'sqlpass123')
}

# 创建连接
mysql_conn = mysql.connector.connect(**mysql_config)
print("MySQL 连接成功！")

# 查看数据库中的表
cursor = mysql_conn.cursor()
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print(f"\n数据库中的表: {len(tables)} 个")
for table in tables[:10]:  # 显示前10个
    print(f"  - {table[0]}")

cursor.close()
mysql_conn.close()

## 3. PostgreSQL 连接示例

In [ ]:
# PostgreSQL 连接配置
postgres_config = {
    'host': os.getenv('POSTGRES_HOST', 'postgres-db'),
    'port': int(os.getenv('POSTGRES_PORT', 5432)),
    'database': os.getenv('POSTGRES_DATABASE', 'sqlExpert'),
    'user': os.getenv('POSTGRES_USERNAME', 'postgres'),
    'password': os.getenv('POSTGRES_PASSWORD', 'postgres123')
}

# 创建连接
postgres_conn = psycopg2.connect(**postgres_config)
print("PostgreSQL 连接成功！")

# 查看数据库中的表
cursor = postgres_conn.cursor()
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'
    ORDER BY table_name
""")
tables = cursor.fetchall()
print(f"\n数据库中的表: {len(tables)} 个")
for table in tables[:10]:  # 显示前10个
    print(f"  - {table[0]}")

cursor.close()
postgres_conn.close()

## 4. 使用 SQLAlchemy + Pandas 查询数据

In [ ]:
# 创建 MySQL SQLAlchemy 引擎
mysql_engine = create_engine(
    f"mysql+pymysql://{mysql_config['user']}:{mysql_config['password']}"
    f"@{mysql_config['host']}:{mysql_config['port']}/{mysql_config['database']}"
)

# 执行 SQL 查询并转换为 DataFrame
# 示例：查询表结构（请根据实际情况修改表名）
query = """
SELECT TABLE_NAME, TABLE_ROWS, DATA_LENGTH, INDEX_LENGTH
FROM information_schema.TABLES
WHERE TABLE_SCHEMA = 'sqlExpert'
ORDER BY TABLE_ROWS DESC
LIMIT 10
"""

df = pd.read_sql(query, mysql_engine)
df

## 5. 数据可视化示例  

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体（如果需要）
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 示例：绘制表大小分布（如果有数据）
if not df.empty and 'TABLE_ROWS' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df.head(10), x='TABLE_NAME', y='TABLE_ROWS')
    plt.title('表行数统计（Top 10）')
    plt.xlabel('表名')
    plt.ylabel('行数')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()